<a href="https://colab.research.google.com/github/ephalt/dog-breed-vision-classification/blob/main/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-end Multi-class Dog Breed Classification using Tensorflow and Tensorflow Hub.

## Problem

Identifying the breed of a dog given an image of a dog.

## Data

The data we are using is from Kaggle's dog breed identification competition.

https://www.kaggle.com/competitions/dog-breed-identification/data

## Evaluation

The evaluation is a file with prediction probabilities for each dog breed of each test image.

https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

## Features

Some information about the data:
* We are dealing with images (unstructured data) so it's probably best we use deep learning/transfer learning.
* There are 120 breeds of dogs (this means there are 120 different classes).
* There are around 10,000+ images in the training set (these images have labels).
* There are around 10,000+ images in the test set (these images have no labels, because we'll want to predict them).

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
# unzip the uploaded data into Google Drive
# !unzip "/content/drive/MyDrive/Dog Vision/dog-breed-identification.zip" -d "drive/MyDrive/Dog Vision/"

# Get our workspace ready

In [ ]:
import tensorflow as tf
print ("TF version:", tf.__version__)

TF version: 2.12.0


In [ ]:
# Import necessary tools:

import tensorflow_hub as hub
print ("TF Hub version:", hub.__version__)

# Check for GPU availability
print("GPU", "available (yes)" if tf.config.list_physical_devices("GPU") else "not available")

TF Hub version: 0.13.0
GPU available (yes)
